<a href="https://colab.research.google.com/github/omarsayed7/RSNA-Pneumonia-Detection/blob/master/RSNA_Pneumonia_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [9]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"omarsayed7","key":"381fb01dc8dc1a948731f4a326853459"}'}

In [10]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [0]:
! cp kaggle.json ~/.kaggle/

In [0]:
! chmod 600 ~/.kaggle/kaggle.json

In [13]:
!kaggle competitions download -c rsna-pneumonia-detection-challenge

  0% 0.00/106k [00:00<?, ?B/s]
100% 106k/106k [00:00<00:00, 42.4MB/s]
  0% 0.00/158k [00:00<?, ?B/s]
100% 158k/158k [00:00<00:00, 50.2MB/s]
  0% 0.00/109k [00:00<?, ?B/s]
100% 109k/109k [00:00<00:00, 87.4MB/s]
  0% 0.00/169k [00:00<?, ?B/s]
100% 169k/169k [00:00<00:00, 49.9MB/s]
  0% 0.00/146k [00:00<?, ?B/s]
100% 146k/146k [00:00<00:00, 48.8MB/s]
  0% 0.00/149k [00:00<?, ?B/s]
100% 149k/149k [00:00<00:00, 47.0MB/s]
  0% 0.00/151k [00:00<?, ?B/s]
100% 151k/151k [00:00<00:00, 49.8MB/s]
  0% 0.00/133k [00:00<?, ?B/s]
100% 133k/133k [00:00<00:00, 136MB/s]
  0% 0.00/153k [00:00<?, ?B/s]
100% 153k/153k [00:00<00:00, 46.4MB/s]
  0% 0.00/116k [00:00<?, ?B/s]
100% 116k/116k [00:00<00:00, 117MB/s]
  0% 0.00/110k [00:00<?, ?B/s]
100% 110k/110k [00:00<00:00, 115MB/s]
  0% 0.00/119k [00:00<?, ?B/s]
100% 119k/119k [00:00<00:00, 124MB/s]
  0% 0.00/115k [00:00<?, ?B/s]
100% 115k/115k [00:00<00:00, 116MB/s]
  0% 0.00/103k [00:00<?, ?B/s]
100% 103k/103k [00:00<00:00, 107MB/s]
  0% 0.00/122k [00:00<?, ?